In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
# from sqlalchemy import create_engine, inspect, func

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# inspector = inspect(engine)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Perform a query to retrieve the date and precipitation scores
results=session.query(Measurement.prcp, Measurement.date).\
    filter(Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()

results


In [ ]:
# Calculate the date 1 year ago from the last data point in the database
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df1 = pd.DataFrame(results, columns=['prcp', 'date'])
df1.set_index('date', inplace=True, )


# Sort the dataframe by date
df1.sort_values('date').head()

In [ ]:
date = [results[1] for result in results]
precp = [results[2] for result in results]

In [ ]:
df1.plot(figsize=(7,5),color="green", title="Precipitation by Date")
index = np.arange(len(precp))
plt.xticks([])
plt.tight_layout()
plt.show()
#savefig

![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df1.describe()


![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_ct=session.query(Station).count()
print(station_ct)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
act_station= session.query(Measurement.station, func.count(Measurement.station)).\
   group_by(Measurement.station).\
   order_by((func.count(Measurement.station)).desc()).all()
print(act_station)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature at the most active station?

rank_station = session.query(Measurement.station,\
               func.min(Measurement.tobs),\
               func.max(Measurement.tobs),\
               func.avg(Measurement.tobs)).group_by(Measurement.station).\
               filter(Measurement.station == 'USC00519281').all()
print(rank_station)

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
alpha_station=session.query(Measurement.tobs).\
        filter(Measurement.station=='USC00519281').\
        filter(Measurement.date >= query_date).all()

print(alpha_station)

In [ ]:
df2 = pd.DataFrame(alpha_station, columns=['tobs'])
#df2.set_index(index, inplace=True, )


# Sort the dataframe by date
df2.head()

In [ ]:
# use .hist to plot histogram
# keep putting in the bin into the picture looks like the example
# looks like 12 might be per each month so bins=12 makes sense
alpha_station_plot = df2['tobs'].hist(bins=12, color='green')
alpha_station_plot.set_title('Temperatures from USC00519281 Station')
alpha_station_plot.set_xlabel('Temperature')
alpha_station_plot.set_ylabel('Frequency')
#add legend
plt.legend(["Temperature"],loc= 'best', prop={'size': 10})
plt.show()
#fig.savefig("Images/Histograph.png")

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
start_date=2010,5,31
end_date=2010,6,5

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
def calc_temps(start_date, end_date):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('2010-05-31', '2012-06-05'))


In [ ]:
trip=calc_temps('2010-05-31', '2010-06-05')
trip

In [ ]:
df3 = pd.DataFrame(trip,columns=['min','max','avg'])
#df2.set_index(index, inplace=True, )


# Sort the dataframe by date
df3.head()

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
df3.plot.bar(color="green",  align='center', width=0.3,title="Trip AVG Temp")
#index = np.arange(len(trip))
#plt.xticks(index)
plt.xlabel('Trip Averages')
plt.ylabel('Temperatures')

plt.show()



In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
rain_fall=session.query(Measurement.date >= start_date).filter(Measurement.date <= end_date).\
        filter(Measurement.tobs).\
        filter(Measurement.station).all()
rain_fall

# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
